In [ ]:
%reload_ext autoreload
%autoreload 2
from importlib import reload

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import h5py
import matplotlib as mpl
from tqdm import tqdm


from holodeck import plot, detstats
import holodeck.anisotropy as anis
from holodeck.constants import YR, MSOL, MPC
import holodeck as holo
import healpy as hp

import hasasia.sim as hsim
import os


In [ ]:
SHAPE = None
NREALS = 500
# NREALS = 20
NFREQS = 40
NLOUDEST = 1000
BGL = 1
VAR_HARD_TIME = 0

BUILD_ARRAYS = False
SAVEFIG = True
TOL=0.01
MAXBADS=5

NVARS = 3
# NVARS = 6

NPSRS = 40
NSKIES = 100
# NSKIES = 15

SHOW_GW=True
ALL_TARGETS = ['hard_time', 'gsmf_phi0', 'gsmf_mchar0_log10', 'mmb_mamp_log10', 'mmb_scatter_dex', 'hard_gamma_inner']

LMAX = 8
NSIDE = 8
PARVARS = [0,1,2]

In [ ]:
cmap_Blues = plot.truncate_colormap('Blues', 0.4, 1)
cmap_Greens = plot.truncate_colormap('Greens', 0.4, 1)
cmap_Oranges = plot.truncate_colormap('Oranges', 0.4, 1)
cmap_Purples = plot.truncate_colormap('Purples', 0.4, 1)
cmap_Greys = plot.truncate_colormap('Greys', 0.4, 0.9)

In [ ]:
def draw_anis_vars(ax, target, xx, yy_cl, params_cl, colors, parvars):
    ll=1
    handles = []
    for vv, var in enumerate(parvars):
        yy = yy_cl[vv][:,:,ll]/yy_cl[vv][:,:,0] # F,R
        pp = 68
        med, *conf = np.percentile(yy, (50, 50-pp/2, 50+pp/2), axis=-1)
        hh, = ax.plot(xx, med, label=params_cl[vv][target], color=colors[vv])
        handles.append(hh)
        ax.fill_between(xx, *conf, alpha=0.25, color=colors[vv])
    return handles
    

In [ ]:
Cl_nihan = np.array([
    [0.20216773, 0.14690035, 0.09676646, 0.07453352, 0.05500382, 0.03177427],
    [0.21201336, 0.14884939, 0.10545698, 0.07734305, 0.05257189, 0.03090662],
    [0.20840993, 0.14836757, 0.09854803, 0.07205384, 0.05409881, 0.03305785],
    [0.19788951, 0.15765126, 0.09615489, 0.07475364, 0.0527356 , 0.03113331],
    [0.20182648, 0.14745265, 0.09681202, 0.0746824 , 0.05503161, 0.0317012 ]])
print(Cl_nihan.shape)

freq_bins_Hz = np.array([2.0, 4.0, 5.9, 7.9, 9.9]) *10**-9 # Hz

xx_nihan = freq_bins_Hz * YR # Hz
color_nihan = '#845b98'

In [ ]:
BUILD_ARRAYS=False
INCLUDE_LIMS = True

xlabel = plot.LABEL_GW_FREQUENCY_YR
xlabel2 = plot.LABEL_GW_FREQUENCY_NHZ
ylabel = r"$C_{\ell>0}/C_0$"
colors_purple = cmap_Purples(np.linspace(0,1,6))
colors_greys = cmap_Greys(np.linspace(0,1,3))
parvars = [0,10,20]

fig, ax = plot.figax_single()
# fig.text(0.55, 0.08, xlabel, va='center', ha='center')
# fig.text(0.55, 1.02, xlabel2, va='center', ha='center')
ax.set_xlabel(xlabel)
ax.set_ylabel(ylabel)


target = 'gsmf_phi0'


colors = cmap_Greens(np.linspace(0,1,len(parvars)))
ll=1

yy_cl, xx_fobs, params_cl = detstats.get_anis_freq_arrays(target, parvars=PARVARS, nloudest=NLOUDEST,
                                                            var_hard_time=VAR_HARD_TIME)
xx = xx_fobs*YR
handles = []
for vv, var in enumerate(parvars):
    yy = yy_cl[vv][:,:,ll]/yy_cl[vv][:,:,0] # F,R
    pp = 68
    med, *conf = np.percentile(yy, (50, 50-pp/2, 50+pp/2), axis=-1)
    hh, = ax.plot(xx, med, label=params_cl[vv][target], color=colors[vv], zorder=6-vv)
    handles.append(hh)
    ax.fill_between(xx, *conf, alpha=0.25, color=colors[vv], zorder=3-vv)

# add GW Only
yy_cl, _, params_cl = detstats.get_anis_freq_arrays(target, parvars=[0,1,2], nloudest=NLOUDEST,
                                                    var_hard_time=None, gw_only=True)
for vv, var in enumerate(parvars):
    yy = yy_cl[vv][:,:,ll]/yy_cl[vv][:,:,0] # F,R
    pp = 68
    med, *conf = np.percentile(yy, (50, 50-pp/2, 50+pp/2), axis=-1)
    ax.plot(xx, med, color=colors_greys[vv], zorder=6-vv, linestyle='--', lw=3)

if INCLUDE_LIMS:
    for ll, ell in enumerate(np.arange(1,7)):
        ax.plot(xx_nihan, Cl_nihan[:,ll], marker='o', linestyle=':',
                color=colors_purple[ll], markersize=4, zorder=3.5)

    ax.legend(handles=handles, loc='lower right', ncols=3, title = plot.PARAM_KEYS[target],
              labelspacing=0.1, borderpad=0.1, columnspacing=0.3, handletextpad=0.1, frameon=False, alignment='right')
    ax.tick_params(axis='y', which='both', right=True, left=True, direction='in')
    # ax.tick_params(axis='x', which='both', top=True, direction='in')       
    # ax.text(0.01,0.99, plot.PARAM_KEYS[targets[ii]], transform = ax.transAxes, va='top', ha='left')

tw = ax.twiny()
tw.grid(False)
xlim = np.array(ax.get_xlim()) / YR * 1e9
tw.set(xlim=xlim, xscale=ax.get_xscale())

    
# fig.subplots_adjust(wspace=0, hspace=0)

In [ ]:
BUILD_ARRAYS=False
INCLUDE_LIMS = True

xlabel = plot.LABEL_GW_FREQUENCY_YR
xlabel2 = plot.LABEL_GW_FREQUENCY_NHZ
ylabel = r"$C_{\ell>0}/C_0$"
targets = [
    'gsmf_phi0', 'gsmf_mchar0_log10', 
      'mmb_mamp_log10',  'mmb_scatter_dex',
      'hard_time',  'hard_gamma_inner']
cmaps = [
    cmap_Greens, cmap_Greens,
    cmap_Oranges, cmap_Oranges,
    cmap_Blues, cmap_Blues
]
colors_purple = cmap_Purples(np.linspace(0,1,6))
colors_greys = cmap_Greys(np.linspace(0,1,3))
parvars = [0,10,20]

fig, ax = plot.figax_single()
# fig.text(0.55, 0.08, xlabel, va='center', ha='center')
# fig.text(0.55, 1.02, xlabel2, va='center', ha='center')
ax.set_xlabel(xlabel)
ax.set_ylabel(ylabel)


target = 'hard_gamma_inner'
yy_cl, xx_fobs, params_cl = detstats.get_anis_freq_arrays(target, parvars=PARVARS, nloudest=NLOUDEST,
                                                            var_hard_time=VAR_HARD_TIME)

xx = xx_fobs*YR

colors = cmap_Blues(np.linspace(0,1,len(parvars)))
ll=1
handles = []
for vv, var in enumerate(parvars):
    yy = yy_cl[vv][:,:,ll]/yy_cl[vv][:,:,0] # F,R
    pp = 68
    med, *conf = np.percentile(yy, (50, 50-pp/2, 50+pp/2), axis=-1)
    hh, = ax.plot(xx, med, label=params_cl[vv][target], color=colors[vv], zorder=6-vv)
    handles.append(hh)
    ax.fill_between(xx, *conf, alpha=0.25, color=colors[vv], zorder=3-vv)

# add GW Only
yy_cl, _, params_cl = detstats.get_anis_freq_arrays('gsmf_phi0', parvars=[0,1,2], nloudest=NLOUDEST,
                                                    var_hard_time=None, gw_only=True)
for vv, var in enumerate(parvars):
    if vv == 1:
        yy = yy_cl[vv][:,:,ll]/yy_cl[vv][:,:,0] # F,R
        pp = 68
        med, *conf = np.percentile(yy, (50, 50-pp/2, 50+pp/2), axis=-1)
        ax.plot(xx, med, color=colors_greys[vv], zorder=6-vv, linestyle='--', lw=3)

if INCLUDE_LIMS:
    for ll, ell in enumerate(np.arange(1,7)):
        ax.plot(xx_nihan, Cl_nihan[:,ll], marker='o', linestyle=':',
                color=colors_purple[ll], markersize=4, zorder=3.5)

    ax.legend(handles=handles, loc='lower right', ncols=3, title = plot.PARAM_KEYS[target],
              labelspacing=0.1, borderpad=0.1, columnspacing=0.3, handletextpad=0.1, frameon=False, alignment='right')
    ax.tick_params(axis='y', which='both', right=True, left=True, direction='in')
    # ax.tick_params(axis='x', which='both', top=True, direction='in')       
    # ax.text(0.01,0.99, plot.PARAM_KEYS[targets[ii]], transform = ax.transAxes, va='top', ha='left')

tw = ax.twiny()
tw.grid(False)
xlim = np.array(ax.get_xlim()) / YR * 1e9
tw.set(xlim=xlim, xscale=ax.get_xscale())

    
# fig.subplots_adjust(wspace=0, hspace=0)

In [ ]:
print('           /Users/emigardiner/GWs/holodeck/output/anatomy_redz/hard_gamma_inner_v3_r500_shapeNone_vtau1')
data1, params1 = detstats.get_data('hard_gamma_inner', nloudest=1000, dets=False, nvars=3,
                                 var_hard_time=None)
data2, params2 = detstats.get_data('hard_gamma_inner', nloudest=1000, dets=False, nvars=3,
                                 var_hard_time=0)


In [ ]:
for par in params2:
    print(par['hard_time'])